## AI-powered Contact Center Intake (AICCI) Agents - Guidelines Design

 * Created:   Feb 2025

In [1]:
from dotenv import load_dotenv
load_dotenv() 
import numpy as np
import pandas as pd
import json
import os
import openai
import random
import time
from datetime import datetime
from openai import AzureOpenAI
import psycopg
from sentence_transformers import SentenceTransformer
import autogen
from autogen import GroupChat
from autogen import GroupChatManager
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen import ConversableAgent, UserProxyAgent, config_list_from_json
from autogen.retrieve_utils import TEXT_FORMATS
import psycopg2
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector
from autogen import AssistantAgent, ConversableAgent, UserProxyAgent
import re

/usr/local/Cellar/jupyterlab/4.2.1/libexec/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


In [2]:
llm_config = {
    "config_list": [
        {
            "model": "gpt-4o",
            "api_key": os.getenv("OPENAI_API_KEY"),
            "api_type": "azure",
            "base_url": os.getenv("OPENAI_API_BASE"),
            "api_version": os.getenv("API_VERSION"),
        },
    ],
    "temperature": 0.0,
    "timeout": 300,
}

In [3]:
load_dotenv()  

api_key = os.getenv("OPENAI_API_KEY")
azure_endpoint = os.getenv("OPENAI_API_BASE")
api_version = os.getenv("API_VERSION")
model = "gpt-4o"
temprature = 0.0
top_p = 1.0

def get_response_client(template, text, temprature=temprature, top_p=top_p, model=model, azure_endpoint=azure_endpoint, api_key=api_key, api_version=api_version):
    time.sleep(1)

    client = AzureOpenAI(
        azure_endpoint = azure_endpoint, 
        api_key= api_key,  
        api_version= api_version
    )

    conversation = [
        {"role": "system", "content": template},
        {"role": "user", "content": text},
    ]

    response = client.chat.completions.create(
        model=model,
        temperature=temprature,
        top_p= top_p,
        max_tokens=4000,
        response_format={ "type": "json_object" },  #JSON mode
        messages=conversation
    )

    return response.choices[0].message.content

In [4]:
# #print(os.getcwd())
# file_path = "/Users/kylezarif/Documents/Github/datascience/Agentic_AI/AutoGen/autogen/HighVolClients.csv"
# df = pd.read_csv(file_path)
# print(df.head())

In [5]:
# df_base = df.groupby('ClientName').agg({
#     'Guidelines': lambda x: ' '.join(x),
#     'CustomFile': 'first'
# }).reset_index()

# print(df_base)

In [6]:
# pd.set_option('display.max_colwidth', None)  # Set to None to display all text in a column

# test = df_base[df_base['ClientName'] == 'Amazon']
# display(test['Guidelines'])

In [7]:
# Option1: BeautifulSoup 'html.parser'
# Option2: Gen AI tagging

In [8]:
html_dirty = """
<html> <head> <title>Amazon Corporate</title> <!-- <link rel="stylesheet" type="text/css" href="ciCC.css" /> --> <script type="text/javascript"> function showhide(num){ var rows = 1; // Number of rows to show/hide //; for(i=1; i<=rows; i++){ which = document.getElementById('row'+i); if(num == i){ which.style.display = (which.style.display=='block') ? 'none':'block'; } else{which.style.display = 'none'}; } } </script>  <script type="text/javascript">     function expander(id) {         if (document.getElementById(id).style.display === "none") {             document.getElementById(id).style.display = "block";             // Custom containers             if ((id === "CustomSHOWYES1") && (document.getElementById("CustomSHOWNO1").style.display === "block")) {                 document.getElementById("CustomSHOWNO1").style.display = "none";             }             if ((id === "CustomSHOWNO1") && (document.getElementById("CustomSHOWYES1").style.display === "block")) {                 document.getElementById("CustomSHOWYES1").style.display = "none";             }             if ((id === "CustomSHOWYES2") && (document.getElementById("CustomSHOWNO2").style.display === "block")) {                 document.getElementById("CustomSHOWNO2").style.display = "none";             }             if ((id === "CustomSHOWNO2") && (document.getElementById("CustomSHOWYES2").style.display === "block")) {                 document.getElementById("CustomSHOWYES2").style.display = "none";             }          } else {             document.getElementById(id).style.display = "none";         }     } </script> <style>     ul {         margin: 10px 0;     } </style> </head> <body> <ul style="list-style:disc">     <!-- FIRST YES OR NO -->     <li><span id="firstYES" style="color:#0000f1;font-weight:bold;text-decoration:underline;cursor: pointer;" onclick="expander('firstSHOW'); expander('firstSHOWYES');">YES</span> / <span id="firstNO" style="color:red;font-weight:bold;text-decoration:underline;cursor: pointer;" onclick="expander('firstSHOW'); expander('firstSHOWNO');">NO</span></li>     <ul id="firstSHOW" style="list-style:disc; display: none;">         <!-- CUSTOM NEW STUFF -->             <li style="display: none;" id="firstSHOWYES">Do you work (or did you work) for a DSP Company (or Business) that delivers packages for Amazon? (DSP driver) <span style="color:#0000f1;font-weight:bold;text-decoration:underline;cursor: pointer;" onclick="expander('CustomSHOWYES1')">YES</span> / <span style="color:red;font-weight:bold;text-decoration:underline;cursor: pointer;" onclick="expander('CustomSHOWNO1')">NO</span></li>             <li style="display: none;" id="firstSHOWNO">Do you deliver packages on road for Amazon either through the DSP, FLEX, HUB Program and Walker Program? <span style="color:#0000f1;font-weight:bold;text-decoration:underline;cursor: pointer;" onclick="expander('CustomSHOWYES1')">YES</span> / <span style="color:red;font-weight:bold;text-decoration:underline;cursor: pointer;" onclick="expander('CustomSHOWNO1')">NO</span></li>             <ul id="CustomSHOWYES1" style="list-style:disc; display: none;">                 <li style="color:#006600;"><span style="background:#006600;color:white;font-weight:bold;">READ:</span> Before we begin, please know that you will need to remain on the line until we've completed the entire interview process to ensure Amazon is able to address your concern as warranted. At the end of this process, I will provide you with a report key so you may follow up on this report. Do you wish to proceed?</li>                 <li>Click <a href="https://secure.ethicspoint.com/domain/en/callcenter_report_custom.asp?clientid=65221" rel="newwindow" target="_blank">here</a> to file a report under the <strong>Last Mile Drivers</strong> program.</li>             </ul>             <ul id="CustomSHOWNO1" style="list-style:disc; display: none;">                 <li>Do you or have you ever delivered packages for Amazon? <span style="color:#0000f1;font-weight:bold;text-decoration:underline;cursor: pointer;" onclick="expander('CustomSHOWYES2')">YES</span> / <span style="color:red;font-weight:bold;text-decoration:underline;cursor: pointer;" onclick="expander('CustomSHOWNO2')">NO</span></li>             </ul>             <ul id="CustomSHOWYES2" style="list-style:disc; display: none;">                 <li style="color:#006600;"><span style="background:#006600;color:white;font-weight:bold;">READ:</span> Before we begin, please know that you will need to remain on the line until we've completed the entire interview process to ensure Amazon is able to address your concern as warranted. At the end of this process, I will provide you with a report key so you may follow up on this report. Do you wish to proceed?</li>                 <li>Click <a href="https://secure.ethicspoint.com/domain/en/callcenter_report_custom.asp?clientid=65221" rel="newwindow" target="_blank">here</a> to file a report under the <strong>Last Mile Drivers</strong> program.</li>             </ul>             <ul id="CustomSHOWNO2" style="list-style:disc; display: none;">                 <li style="color:#006600;"><span style="background:#006600;color:white;font-weight:bold;">READ:</span> Before we begin, please know that you will need to remain on the line until we've completed the entire interview process to ensure Amazon is able to address your concern as warranted. At the end of this process, I will provide you with a report key so you may follow up on this report. Do you wish to proceed?</li>                 <br>                 <li><span style="background:red;color:white;font-weight:bold;">DON'T READ:</span> If the caller has reached the correct line, click <a href="https://secure.ethicspoint.com/domain/en/callcenter_report_custom.asp?clientid=44171" rel="newwindow" target="_blank">here</a> to file a new report.</li>             </ul>         <br>     </ul> </ul> <p><span style="background:red;color:white;font-weight:bold;">NOTE:</span> Amazon accepts reports from drivers who deliver Amazon packages.  Proceed with those calls.</p> </body> </html>
"""

In [9]:
template_eda_guidelines = """
Role: Assistant to extract text from HTML to JSON.
Objective: Extract text in JSON from text or HTML elements to a valid JSON.
Make sure the YES / NO is printed after the assocaited question.
Extract text in JSON from text or HTML elements similar to the following format in a valid JSON. 

{ "DONT READ": "Instructions or notes meant for the AI agent or customer service representative that should not be read aloud to the caller.", "READ": "Scripted statements or questions that should be read aloud to the caller.", "Instructions": "General guidelines or procedures that the AI agent or customer service representative should follow during the call.", "Follow-Up Report Instructions": "Specific steps to follow when handling follow-up reports, including what to say to the caller and how to process the report.", "Frequently Asked Questions (FAQ)": "Common questions and answers related to reporting, company policies, and procedures.", "Redirects": "Information on who to contact or where to direct the caller for specific requests or issues that are outside the scope of the current call.", "Partial Reports Instructions": "Guidelines for handling and submitting partial reports, including any special procedures or passwords required.", "Company List": "A list of companies or subsidiaries that are relevant to the reporting process.", "Important Notes": "Critical information that needs special attention, such as changes in service providers or specific instructions for certain types of reports.", "Customer Service Notes (CS NOTES)": "Additional instructions or notes for customer service representatives to follow during the call.", "Questions": "Custom questions that the AI agent should ask the caller to gather necessary information for the report.", "Clarification about reports to file or not file": "Guidance on whether a report should be filed based on the caller's information or situation.", "Location selection": "Instructions or options for selecting the relevant location for the report.", "Case questions": "Specific questions related to the details of the case being reported.", "Involved parties": "Information or questions about the individuals or entities involved in the report.", "Closing": "Instructions or statements for concluding the call or report.", "Closing messages": "Scripted messages to be read at the end of the call.", "Report follow-up info": "Information provided to the caller about how they can follow up on their report." } 

Do not wrap the output in quotation marks.
Do not wrap the output in code block delimiters (```).
"""

guidelines = get_response_client(template_eda_guidelines, str(html_dirty))
guidelines = json.loads(guidelines)
print(guidelines)

{'Questions': [{'question': 'Do you work (or did you work) for a DSP Company (or Business) that delivers packages for Amazon? (DSP driver)', 'options': ['YES', 'NO']}, {'question': 'Do you deliver packages on road for Amazon either through the DSP, FLEX, HUB Program and Walker Program?', 'options': ['YES', 'NO']}, {'question': 'Do you or have you ever delivered packages for Amazon?', 'options': ['YES', 'NO']}], 'READ': ["Before we begin, please know that you will need to remain on the line until we've completed the entire interview process to ensure Amazon is able to address your concern as warranted. At the end of this process, I will provide you with a report key so you may follow up on this report. Do you wish to proceed?"], 'DONT READ': ['If the caller has reached the correct line, click here to file a new report.'], 'Important Notes': ['Amazon accepts reports from drivers who deliver Amazon packages. Proceed with those calls.'], 'Redirects': ['Click here to file a report under the

In [10]:
# with open("guidelines.json", "w") as json_file:
#     json.dump(guidelines, json_file)

# def load_data():
#     with open("guidelines.json", "r") as json_file:
#         data = json.load(json_file)
#     return data

# guidelines = load_data()
# print("Guidelines:", guidelines)

In [11]:
template_memory = """
Role: You are the JSON Chat Memory Agent.
Objective: Your primary responsibility is to memorize the new message and the {{JSON Chat Memory}}.
You will take to action!

Do not wrap the output in quotation marks.
Do not wrap the output in code block delimiters (```).
"""

template_router = """
Role: You are a classification agent responsible for managing the routing of new messages based on predefined rules and priorities.
Objective: You will analyze the incoming message, evaluate the associated {{JSON Chat Memory}}, consider the relevant {{group_chat}} priorities, 
and apply the applicable {{termination_requirements}}. Your task is to:
Detect the correct action ID: Based on the new message, refer to the {{JSON Chat Memory}} to determine the appropriate action ID that aligns with the {{termination_requirements}}.
Route the message to the appropriate group: After determining the action id, select the next appropriate {{group_chat}} for the message from the available options.

{
  "group_chat": 
  [
   
    {"id": "main_guidelines",
    "priority": 2, 
    "role": "This function is used when the JSON Chat Memory does not contain the customer's responses to the READ messages in guidelines/instructions.",
    "termination_requirements": {
        "requirement_1": "Have answers to all READ messages.",
        "requirement_2": "Have followed instructions for NOT READ messages",
        "requirement_3": "Have identified the caller's intent to determine whether it is a new report or a follow-up.",
        "requirement_4": "Take action by using procedures. Complete the procedures in the instructions, whether it is a new report or a follow-up."
      }
    }
    
  ]
}


example: 

{"action":"main_guidelines"}

Do not wrap the output in quotation marks.
Do not wrap the output in code block delimiters (```).
"""

In [12]:
with open("memory.json", "w") as json_file:
    json.dump([{}], json_file)

def load_chat_history():
    with open("memory.json", "r") as json_file:
        chat_history_data = json.load(json_file)
    return chat_history_data

chat_history = load_chat_history()
print("Initial chat history:", chat_history)

Initial chat history: [{}]


In [13]:
memory_agent = ConversableAgent(
    name="memory_agent",
    system_message= f"{template_memory} + chat history: {chat_history}",
    llm_config=llm_config,
    human_input_mode= "NEVER",
)

router_agent = ConversableAgent(
    name="router_agent",
    system_message= f"{template_router} + guidelines instructions: {guidelines}",
    llm_config=llm_config,
    human_input_mode= "NEVER",
)

In [14]:
def state_transition_router(last_speaker, groupchat):
    messages = groupchat.messages

    if last_speaker is memory_agent:
        return router_agent
    elif last_speaker is router_agent:
        return None

groupchat_router = autogen.GroupChat(
    agents=[memory_agent,
            router_agent
           ],
    messages=[],
    max_round=10,
    speaker_selection_method=state_transition_router,
)

manager_router = autogen.GroupChatManager(groupchat=groupchat_router, llm_config=llm_config)

In [15]:
template_guidelines = """
Role: Ethics and Compliance Contact Center Analyst
Objective: Handle calls by following a structured process using provided guidelines and previous chat history.

Steps:

Start with the "OPENING" statement from the guidelines if available.
if not:

OPENING = 'Thanks you for calling contact center. This is AICCI.'

Sequentially address each "READ" message: 
a. If not addressed, respond with the "READ" message. 
b. If addressed, move to the next "READ" message.
Determine if the call is a "Follow-Up" or "New Report": 
a. For "Follow-Up", follow "Other_Instructions" if available and respond with the "READ" message is "affiliation"  
b. For "New Report", follow "Other_Instructions" if available.
Continue addressing each "READ" message in sequence.

Example 1: New Report

"OPENING": "Text from opening HTML element"
"READ_1": "1st Text to read"
"READ_2": "2nd Text to read"

Do not wrap the output in quotation marks.
Do not wrap the output in code block delimiters (```).
"""

In [16]:
user_agent = ConversableAgent(
    name="user_agent",
    system_message= f"You are an assistant",
    llm_config=llm_config,
    human_input_mode= "NEVER",
)

guidelines_agent = ConversableAgent(
    name="guidelines_agent",
    system_message= f"guidelines: {guidelines} + {template_guidelines}",
    llm_config=llm_config,
    human_input_mode= "NEVER",
)

In [17]:
def state_transition_guidelines(last_speaker, groupchat):
    messages = groupchat.messages

    if last_speaker is user_agent:
        return guidelines_agent
    elif last_speaker is guidelines_agent:
        return None

groupchat_guidelines = autogen.GroupChat(
    agents=[user_agent,
            guidelines_agent
           ],
    messages=[],
    max_round=10,
    speaker_selection_method=state_transition_guidelines,
)

manager_guidelines = autogen.GroupChatManager(groupchat=groupchat_guidelines, llm_config=llm_config)

In [18]:
def collect_json_data():
    follow_ups = {"questions": []}
    while not follow_ups["questions"]:
        user_input = input('Please provide data in JSON format (e.g. {"questions": [{"question": "your question", "answer": "your answer"}]}): ')
        try:
            user_data = json.loads(user_input)
            if isinstance(user_data, dict) and "questions" in user_data and isinstance(user_data["questions"], list):
                valid = all(isinstance(item, dict) and "question" in item and "answer" in item for item in user_data["questions"])
                if valid:
                    follow_ups.update(user_data)
                else:
                    print("Each item in the questions list must be an object with question and answer keys. Please try again.")
            else:
                print("The JSON input must be an object with a questions key containing a list of question-answer objects. Please try again.")
        except json.JSONDecodeError:
            print("Invalid JSON input. Please try again.")
    return follow_ups


def extract_questions_from_memory(memory):
    last_message = memory[-1]
    content = last_message["content"]
    try:
        content_dict = json.loads(content)
        if "questions" in content_dict and isinstance(content_dict["questions"], list):
            return content_dict
        else:
            raise json.JSONDecodeError("Invalid format", content, 0)
    except json.JSONDecodeError:
        questions = [q.strip() + "?" if not q.strip().endswith("?") else q.strip() for q in content.split("\n") if q.strip()]
        questions_dict = {"questions": [{"question": q, "answer": ""} for q in questions]}
        return json.dumps(questions_dict)


def read_chat_history():
    try:
        with open("memory.json", "r") as json_file:
            try:
                return json.load(json_file)
            except json.JSONDecodeError:
                return []  
    except FileNotFoundError:
        return [] 


def extract_and_print_questions():
    memory = read_chat_history()
    questions = extract_questions_from_memory(memory)
    print(questions)


def write_chat_history(chat_history_data):
    seen = set()
    unique_messages = []

    for message in chat_history_data:
        message_str = json.dumps(message, sort_keys=True, ensure_ascii=False)  
        if message_str not in seen:
            seen.add(message_str)
            unique_messages.append(message)

    with open("memory.json", "w", encoding="utf-8") as json_file:
        json.dump(unique_messages, json_file, ensure_ascii=False)

In [19]:
def main_guidelines(new_message):
    memory = read_chat_history()
    memory.append(new_message)

    last_message = json.dumps(new_message)

    user_agent.initiate_chat(recipient=manager_guidelines, message=last_message, clear_history=False)
    messages_json = manager_guidelines.messages_to_string(manager_guidelines.groupchat.messages)

    new_messages = json.loads(messages_json)
    memory.extend(new_messages)

    write_chat_history(memory)
    extract_and_print_questions()

In [20]:
template_report_type_checker= """
Role: classsification agent to determine the call reason.
Objective: determine the call reason, whether it is a "Follow-Up" or a "New Report" in a valid JSON.

Example1:
{"report_type":"New Report"}

Example2:
{"report_type":"Follow-Up"}

Do not wrap the output in quotation marks.
Do not wrap the output in code block delimiters (```).
"""

report_type_checker_agent = ConversableAgent(
    name="report_type_checker_agent",
    system_message= template_report_type_checker,
    llm_config=llm_config,
    human_input_mode= "NEVER",
)

def state_transition_report_type_checker(last_speaker, groupchat):
    messages = groupchat.messages

    if last_speaker is user_agent:
        return report_type_checker_agent
    elif last_speaker is report_type_checker_agent:
        return None

groupchat_report_type_checker = autogen.GroupChat(
    agents=[user_agent,
            report_type_checker_agent
           ],
    messages=[],
    max_round=10,
    speaker_selection_method=state_transition_report_type_checker,
)

manager_report_type_checker = autogen.GroupChatManager(groupchat=groupchat_report_type_checker, llm_config=llm_config)

def main_report_type_checker():
    memory = read_chat_history()
    last_message = json.dumps(memory)

    user_agent.initiate_chat(recipient=manager_report_type_checker, message=last_message, clear_history=False)
    messages_json = manager_report_type_checker.messages_to_string(manager_report_type_checker.groupchat.messages)

    new_messages = json.loads(messages_json)
    report_type_checker = new_messages[-1]
    memory.append(report_type_checker)
    
    write_chat_history(memory)

In [ ]:
def detect_action():
    memory = read_chat_history()
    new_message = collect_json_data()
    memory.append(new_message)

    last_message = json.dumps(memory)
    history = memory_agent.initiate_chat(recipient=manager_router, message=last_message, clear_history=False)
    message = history.chat_history[-1]["content"]
    last_message = json.loads(message)
    action = last_message.get("action", None)

    return action, new_message 

def process_action(action, new_message):
    if action == "main_guidelines":
        return main_guidelines(new_message)
    else:
        print(f"Invalid action: {action}")
        return {"error": "Invalid action"}

if __name__ == "__main__":
    while True:
        action, new_message = detect_action()
        process_action(action, new_message)
        if action == "terminate_chat":
            break

Please provide data in JSON format (e.g. {"questions": [{"question": "your question", "answer": "your answer"}]}):  {"questions": [{"question": "your question", "answer": "your answer"}]}


memory_agent (to chat_manager):

[{}, {"questions": [{"question": "your question", "answer": "your answer"}]}]

--------------------------------------------------------------------------------

Next speaker: router_agent

router_agent (to chat_manager):

{"action":"main_guidelines"}

--------------------------------------------------------------------------------
user_agent (to chat_manager):

{"questions": [{"question": "your question", "answer": "your answer"}]}

--------------------------------------------------------------------------------

Next speaker: guidelines_agent

guidelines_agent (to chat_manager):

Thank you for calling the contact center. This is AICCI.

Before we begin, please know that you will need to remain on the line until we've completed the entire interview process to ensure Amazon is able to address your concern as warranted. At the end of this process, I will provide you with a report key so you may follow up on this report. Do you wish to proceed?

Do you wor

Please provide data in JSON format (e.g. {"questions": [{"question": "your question", "answer": "your answer"}]}):  {"questions": [{"question": "Thank you for calling the contact center. This is AICCI.?", "answer": ""}, {"question": "Before we begin, please know that you will need to remain on the line until we've completed the entire interview process to ensure Amazon is able to address your concern as warranted. At the end of this process, I will provide you with a report key so you may follow up on this report. Do you wish to proceed?", "answer": "Yes please"}, {"question": "Do you work (or did you work) for a DSP Company (or Business) that delivers packages for Amazon? (DSP driver)?", "answer": "Yes"}]}


memory_agent (to chat_manager):

[{}, {"questions": [{"question": "your question", "answer": "your answer"}]}, {"content": "{\"questions\": [{\"question\": \"your question\", \"answer\": \"your answer\"}]}", "role": "user", "name": "user_agent"}, {"content": "Thank you for calling the contact center. This is AICCI.\n\nBefore we begin, please know that you will need to remain on the line until we've completed the entire interview process to ensure Amazon is able to address your concern as warranted. At the end of this process, I will provide you with a report key so you may follow up on this report. Do you wish to proceed?\n\nDo you work (or did you work) for a DSP Company (or Business) that delivers packages for Amazon? (DSP driver)", "role": "user", "name": "guidelines_agent"}, {"questions": [{"question": "Thank you for calling the contact center. This is AICCI.?", "answer": ""}, {"question": "Before we begin, please know that you will need to remain on the line until we've completed 

Please provide data in JSON format (e.g. {"questions": [{"question": "your question", "answer": "your answer"}]}):  {"questions": [{"question": "Do you deliver packages on road for Amazon either through the DSP, FLEX, HUB Program and Walker Program?", "answer": "No"}]}


memory_agent (to chat_manager):

[{}, {"questions": [{"question": "your question", "answer": "your answer"}]}, {"content": "{\"questions\": [{\"question\": \"your question\", \"answer\": \"your answer\"}]}", "role": "user", "name": "user_agent"}, {"content": "Thank you for calling the contact center. This is AICCI.\n\nBefore we begin, please know that you will need to remain on the line until we've completed the entire interview process to ensure Amazon is able to address your concern as warranted. At the end of this process, I will provide you with a report key so you may follow up on this report. Do you wish to proceed?\n\nDo you work (or did you work) for a DSP Company (or Business) that delivers packages for Amazon? (DSP driver)", "role": "user", "name": "guidelines_agent"}, {"questions": [{"question": "Thank you for calling the contact center. This is AICCI.?", "answer": ""}, {"question": "Before we begin, please know that you will need to remain on the line until we've completed 

Please provide data in JSON format (e.g. {"questions": [{"question": "your question", "answer": "your answer"}]}):  {"questions": [{"question": "Do you or have you ever delivered packages for Amazon?", "answer": "No"}]}


memory_agent (to chat_manager):

[{}, {"questions": [{"question": "your question", "answer": "your answer"}]}, {"content": "{\"questions\": [{\"question\": \"your question\", \"answer\": \"your answer\"}]}", "role": "user", "name": "user_agent"}, {"content": "Thank you for calling the contact center. This is AICCI.\n\nBefore we begin, please know that you will need to remain on the line until we've completed the entire interview process to ensure Amazon is able to address your concern as warranted. At the end of this process, I will provide you with a report key so you may follow up on this report. Do you wish to proceed?\n\nDo you work (or did you work) for a DSP Company (or Business) that delivers packages for Amazon? (DSP driver)", "role": "user", "name": "guidelines_agent"}, {"questions": [{"question": "Thank you for calling the contact center. This is AICCI.?", "answer": ""}, {"question": "Before we begin, please know that you will need to remain on the line until we've completed 

Please provide data in JSON format (e.g. {"questions": [{"question": "your question", "answer": "your answer"}]}):  {"questions": [{"question": "If the caller has reached the correct line, click here to file a new report.?", "answer": "I cant click"}]}


memory_agent (to chat_manager):

[{}, {"questions": [{"question": "your question", "answer": "your answer"}]}, {"content": "{\"questions\": [{\"question\": \"your question\", \"answer\": \"your answer\"}]}", "role": "user", "name": "user_agent"}, {"content": "Thank you for calling the contact center. This is AICCI.\n\nBefore we begin, please know that you will need to remain on the line until we've completed the entire interview process to ensure Amazon is able to address your concern as warranted. At the end of this process, I will provide you with a report key so you may follow up on this report. Do you wish to proceed?\n\nDo you work (or did you work) for a DSP Company (or Business) that delivers packages for Amazon? (DSP driver)", "role": "user", "name": "guidelines_agent"}, {"questions": [{"question": "Thank you for calling the contact center. This is AICCI.?", "answer": ""}, {"question": "Before we begin, please know that you will need to remain on the line until we've completed 

In [ ]:
with open("memory.json", "r") as json_file:
    chat_history_guidelines = json.load(json_file)
print(json.dumps(chat_history_guidelines, indent=3))